## import:


In [1]:
#importer la bibliothèque de traitement d'image (opencv)
import cv2

#bibliothèque pour manipuler des matrice
import numpy as np # bibliotheque pour le traitement d'image 

import sys

import os

import random

import mediapipe # biblihotèque pour la reconnaissance faciale  


mp_drawing = mediapipe.solutions.drawing_utils 

## variables


In [2]:
# haar_file = 'haarcascade_frontalface_default.xml'

drawingModule = mediapipe.solutions.drawing_utils 
faceModule = mediapipe.solutions.face_mesh
 
circleDrawingSpec = drawingModule.DrawingSpec(thickness=1, circle_radius=1, color=(0,255,0)) #dans la variable circleDrawingSpec deffinis la couleur et l'épaisseur des point sur le visage
lineDrawingSpec = drawingModule.DrawingSpec(thickness=1, color=(0,255,0)) #dans la variable lineDrawingSpec deffinis la couleur et l'epaisseur d'une ligne des traits sur le visage

compteur = 1

nombreNomImage = 1

#nom du répertoire:

bilal = "image_bilal"
sonia = "image_sonia"
ilyes = "image_ilyes"
yassin = "image_yassin"

#nom utilisateur:

userB = "Bilal-"
userS = "Sonia-"
userI = "Ilyes-"
userY = "Yassin-"

#délais de capture de visage:
# 1 seconde = 24 (24 frame par seconde)
delaisSauvegarde = 24*5

## Reconnaissance d'un visage humain


In [3]:
cap = cv2.VideoCapture(0) # ouvre un canal vidéo de la caméra de mon ordinateur variable="cap"

if not cap.isOpened(): #SI le canal de la vidéo n'est pas fonctionnelle alors
    print("Cannot open camera") # affiche le texte
    exit() # sort du programme entier


while True: #effectue la suite du code en boucle indeffiniment
    # Capture frame-by-frame

    #lis la vidéo variable="codeRetour" et "frame" 
    #la variable "codeRetour" contient sois un succé de lecture de la vidéo soit un echec de lecture de la vidéo
    #la variable "frame" veut dire image
    codeRetourSuccess, frame = cap.read() # lis la vidéo
    # if frame is read correctly ret is True
    if not codeRetourSuccess: #si codeRetourSuccess n'est pas un succé de lecture de la vidéo
        print("Can't receive frame (stream end?). Exiting ...") #affiche le message
        break # sort de la boucle 
    
    framRectangle = frame.copy() # copie la lecture de la vidéo
    imageRgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #convertis l'image bgr en rgb

    visages = []
    
    with faceModule.FaceMesh(static_image_mode=True, max_num_faces=100) as face: #detecte les visages humains (au maximum 100 visage humain)
        results = face.process(imageRgb) # detecte les visages dans l'image de type rgb
    # print(len(results.multi_face_landmarks))

        # h, w, c = frame.shape #shape = dimension de l'image, h=height, w=width, c=color
        # cx_min=  w #affecte la largeur de l'image à la variable cx_min
        # cy_min = h #affecte la hauteur de l'image à la variable cy_min
        # cx_max= cy_max= 0

        if results.multi_face_landmarks != None: #si tu a trouver des visage
            
            for faceLandmarks in results.multi_face_landmarks: # pour chaque visage trouvé effectue la suite
                # drawingModule.draw_landmarks(frame, faceLandmarks, faceModule.FACEMESH_FACE_OVAL, circleDrawingSpec, lineDrawingSpec) #dessine les points et les trais sur le visage 
                
                h, w, c = frame.shape #shape = dimension de l'image, h=height, w=width, c=color
                cx_min=  w #affecte la largeur de l à la variable cx_min
                cy_min = h #affecte 
                cx_max= cy_max= 0
                for id, lm in enumerate(faceLandmarks.landmark): #pour chaque visage trouver effectue la suite
                    # print(id,lm)
                    cx, cy = int(lm.x * w), int(lm.y * h) #affecte la largeur à X et la hauteur à Y
                    if cx<cx_min: #si la largeur du visage est plus petite que la largeur minimum d'un visage 
                        cx_min=cx #la largeur minimum sera égal à la largeur du visage
                    if cy<cy_min: #si la hauteur du visage est plus petite que la hauteur minimum d'un visage 
                        cy_min=cy #la hauteur minimum sera égal à la hauteur du visage
                    if cx>cx_max: #si la largeur du visage est plus grande que la largeur maximum d'un visage
                        cx_max=cx #la largeur maximum sera égal à la largeur du visage
                    if cy>cy_max: ##si la hauteur du visage est plus grande que la hauteur maximum d'un visage
                        cy_max=cy #la hauteur maximum sera égal à la hauteur du visage
                cv2.rectangle(framRectangle, (cx_min, cy_min), (cx_max, cy_max), (255, 255, 0), 2) #dessine le rectangle sur chaque visage
                visages.append((cx_min, cy_min, cx_max, cy_max))
    if compteur == delaisSauvegarde: # si le compteur est égale à 24 effectue la suite
    #coupe ta tête du point A au point B, A = cx_min, cy_min, B = cx_max, cy_max
    # print(compteur)
        compteur = 1 #réinitialise le conteur à 1


        # imageSave = frame[cx_min:cy_min, cx_max:cy_max]
        
        # cv2.imwrite("C:/travail/informatique/intelligence_artificielle/image_save/image"+ str(nombreNomImage) +".jpg", imageSave)
        # cv2.imshow('ma tête', imageSave)
        
        for visagesAB in visages:

            cx_min, cy_min, cx_max, cy_max = visagesAB
            # print(cx_min, cy_min, cx_max, cy_max)
            if cx_min != 640 and cy_min != 480 and cx_max != 0 and cy_max != 0: # si les conditions suivante son respecté effectue la suite, elle sert à couper le visage si il sort du cadre de la vidéo
                # cv2.imwrite("C:/travail/informatique/intelligence_artificielle/image_save/image"+ str(nombreNomImage) +".jpg", imageSave)
                if cx_min < 0: # si la largeur minimum  du visage est inférieur à 0, mets la à 0, 0 = tout à gauche de webcam
                    cx_min = 0
                if cx_max > 640: # si la largeur maximum du visage est supérieur à 640, mets la à 640, 640=tout à droite de la webcam
                    cx_max = 640 
                if cy_min < 0: # si la hauteur minimal du visage est inférieur à 0, mets la à 0, 0 = tout en bas de la webcam
                    cy_min = 0
                if cy_max > 480: #si la hauteur maximal du visage est supérieur à 480, mets la à 480, 480=tout en haut de la webcam
                    cy_max = 480
                imageSave = frame[cy_min:cy_max,cx_min:cx_max] #  capture les contours du visages dans la variable imagesave 

                
                # print(cx_min,cx_max, cy_min,cy_max)
                #sauvegarde l'imagesave de la vidéo à l'instanté dans le répertoire suivant (donc des que le compteur est à 24 pour respecter la condition plus haute):
                randomAleatoir = random.randrange(900000000,999999999)	
                cv2.imwrite("C:/travail/informatique/intelligence_artificielle/" + bilal + "/" + userB + str(randomAleatoir) +".jpg", imageSave) 
                # cv2.imshow('imageSave', imageSave)
                # print(imageSave.shape)


        # print("/image_save/image"+ str(nombreNomImage) +".jpg")
        nombreNomImage = nombreNomImage + 1
    else: #sinon ajoute 1 au compteur jusqu'à qu'il fasse 24 pour rentrer dans la condition
        compteur = compteur + 1
        # print ("sinon")

    # Display the resulting frame
    cv2.imshow('framRectangle', framRectangle) #affiche l'image
    if cv2.waitKey(1) == ord('q'): #si j'appuie  la touche 'q' sort de la boucle donc quitte la vidéo
        break # sort de la boucle

# When everything done, release the capture
cap.release() # libère la mémoire les ressource de la vidéo, des variables... lu par le programme
cv2.destroyAllWindows() # détruit les fenêtres ouverte par le programme (mémoire vidéo)sur

## Reonnaissance visage humain dans une image


In [ ]:
import cv2
import mediapipe

drawingModule = mediapipe.solutions.drawing_utils
faceModule = mediapipe.solutions.face_mesh
 
circleDrawingSpec = drawingModule.DrawingSpec(thickness=1, circle_radius=1, color=(0,255,0))
lineDrawingSpec = drawingModule.DrawingSpec(thickness=1, color=(0,255,0))
 
with faceModule.FaceMesh(static_image_mode=True, max_num_faces=100) as face: #

#---------------------------------------------------------------------------------------------------------------------------
    #variable de l'image 
    image = cv2.imread("Common-face-shape-categories.png")
 
    results = face.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

#---------------------------------------------------------------------------------------------------------------------------
     
    # print(results.count)

    if results.multi_face_landmarks != None:
        for faceLandmarks in results.multi_face_landmarks:
            drawingModule.draw_landmarks(image, faceLandmarks, faceModule.FACEMESH_CONTOURS, circleDrawingSpec, lineDrawingSpec)
 

    # h, w, c = image.shape #shape = dimension de l'image, h=height, w=width, c=color
    # cx_min=  w #affecte la largeur de l'image à la variable cx_min
    # cy_min = h #affecte la hauteur de l'image à la variable cy_min
    # cx_max= cy_max= 0
    if results.multi_face_landmarks != None: #si tu a trouver des visage
                for faceLandmarks in results.multi_face_landmarks: # pour chaque visage trouvé effectue la suite
                    # drawingModule.draw_landmarks(frame, faceLandmarks, faceModule.FACEMESH_FACE_OVAL, circleDrawingSpec, lineDrawingSpec) #dessine les points et les trais sur le visage 
                    
                    h, w, c = image.shape #shape = dimension de l'image, h=height, w=width, c=color
                    cx_min=  w #affecte la largeur de l à la variable cx_min
                    cy_min = h #affecte 
                    cx_max= cy_max= 0
                    for id, lm in enumerate(faceLandmarks.landmark): #pour chaque visage trouver effectue la suite
                        # print(id,lm)
                        cx, cy = int(lm.x * w), int(lm.y * h) #affecte la largeur à X et la hauteur à Y
                        if cx<cx_min: #si la largeur du visage est plus petite que la largeur minimum d'un visage 
                            cx_min=cx #la largeur minimum sera égal à la largeur du visage
                        if cy<cy_min: #si la hauteur du visage est plus petite que la hauteur minimum d'un visage 
                            cy_min=cy #la hauteur minimum sera égal à la hauteur du visage
                        if cx>cx_max: #si la largeur du visage est plus grande que la largeur maximum d'un visage
                            cx_max=cx #la largeur maximum sera égal à la largeur du visage
                        if cy>cy_max: ##si la hauteur du visage est plus grande que la hauteur maximum d'un visage
                            cy_max=cy #la hauteur maximum sera égal à la hauteur du visage
                    cv2.rectangle(image, (cx_min, cy_min), (cx_max, cy_max), (255, 255, 0), 2) #affecte le rectangle 
    
    cv2.imshow('Test image', image)
 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

copie

In [ ]:
h, w, c = image.shape #shape = dimension de l'image, h=height, w=width, c=color
            cx_min=  w #affecte la largeur de l'image à la variable cx_min
            cy_min = h #affecte la hauteur de l'image à la variable cy_min
            cx_max= cy_max= 0
            if results.multi_face_landmarks != None: #si tu a trouver des visage
                for faceLandmarks in results.multi_face_landmarks: # pour chaque visage trouvé effectue la suite
                    # drawingModule.draw_landmarks(frame, faceLandmarks, faceModule.FACEMESH_FACE_OVAL, circleDrawingSpec, lineDrawingSpec) #dessine les points et les trais sur le visage 
                    
                    # h, w, c = frame.shape #shape = dimension de l'image, h=height, w=width, c=color
                    # cx_min=  w #affecte la largeur de l à la variable cx_min
                    # cy_min = h #affecte 
                    # cx_max= cy_max= 0
                    for id, lm in enumerate(faceLandmarks.landmark): #pour chaque visage trouver effectue la suite
                        # print(id,lm)
                        cx, cy = int(lm.x * w), int(lm.y * h) #affecte la largeur à X et la hauteur à Y
                        if cx<cx_min: #si la largeur du visage est plus petite que la largeur minimum d'un visage 
                            cx_min=cx #la largeur minimum sera égal à la largeur du visage
                        if cy<cy_min: #si la hauteur du visage est plus petite que la hauteur minimum d'un visage 
                            cy_min=cy #la hauteur minimum sera égal à la hauteur du visage
                        if cx>cx_max: #si la largeur du visage est plus grande que la largeur maximum d'un visage
                            cx_max=cx #la largeur maximum sera égal à la largeur du visage
                        if cy>cy_max: ##si la hauteur du visage est plus grande que la hauteur maximum d'un visage
                            cy_max=cy #la hauteur maximum sera égal à la hauteur du visage
                    cv2.rectangle(image, (cx_min, cy_min), (cx_max, cy_max), (255, 255, 0), 2) #affecte le rectangle 